In [5]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [6]:
import struct
import os

def read_mnist_images(file_path):
    with open(file_path, 'rb') as f:
        magic, num_images = struct.unpack('>II', f.read(8))
        rows, cols = struct.unpack('>II', f.read(8))
        images = np.fromfile(f, dtype=np.uint8).reshape(num_images, rows * cols)
    return images

def read_mnist_labels(file_path):
    with open(file_path, 'rb') as f:
        magic, num_labels = struct.unpack('>II', f.read(8))
        labels = np.fromfile(f, dtype=np.uint8)
    return labels

# Replace these paths with your actual file paths
train_images_path = 'Dataset/train-images-idx3-ubyte/train-images-idx3-ubyte'  # Update with your path
train_labels_path = 'Dataset/train-labels-idx1-ubyte/train-labels-idx1-ubyte'  # Update with your path
test_images_path = 't10k-images-idx3-ubyte'    # Update with your path
test_labels_path = 't10k-labels-idx1-ubyte'    # Update with your path

# Load data
train_images = read_mnist_images(train_images_path)
train_labels = read_mnist_labels(train_labels_path)

# Create DataFrames
# First for the features (images)
train_df = pd.DataFrame(train_images)
# Add the labels
train_df['label'] = train_labels


In [7]:
df=train_df.copy()
df.head()

,0,1,2,3,4,5,6,7,8,9,...,775,776,777,778,779,780,781,782,783,label
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,5
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,4
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,9


In [25]:
sample_data=df.iloc[0:10000,:]

In [26]:
X=np.array(sample_data.iloc[:,0:784])
y=np.array(sample_data.iloc[:,784])
X

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]], dtype=uint8)

In [27]:
class Layer_Dense:
    def __init__(self,n_inputs,n_neuron):
        self.weights=0.10*np.random.randn(n_inputs,n_neuron)
        self.bias=0.10*np.random.randn(1,n_neuron)
    def forward(self,inputs):
        self.inputs=inputs
        self.output=np.dot(inputs,self.weights)+self.bias
    def backward(self,dvalues):
        self.dweights=np.dot(self.inputs.T,dvalues)
        self.dbias=np.sum(dvalues,axis=0,keepdims=True)
        self.dinputs=np.dot(dvalues,self.weights.T)

class Activation_RelU:
    def forward(self,inputs):
        self.inputs=inputs
        self.output=np.maximum(0,inputs)
    def backward(self,dvalues):
        self.dinputs=dvalues.copy()
        self.dinputs[self.inputs<=0]=0

class Activation_SoftMax:
    def forward(self,inputs):
        self.inputs=inputs
        exp_values=np.exp(inputs-np.max(inputs,axis=1,keepdims=True))
        probablities=exp_values/np.sum(exp_values,axis=1,keepdims=True)
        self.output=probablities

class Loss:
    def calculate(self,output,y):
        sample_losses=self.forward(output,y)
        data_loss=np.mean(sample_losses)
        return data_loss
class Loss_CategoricalCross_Entropy(Loss):
    def forward(self,y_pred,y_true):
        samples=len(y_pred)
        y_pred_clipped=np.clip(y_pred,1e-7,1-1e-7)
        if len(y_true.shape)==1:
            correct_confidence=y_pred_clipped[range(samples),y_true]
        elif len(y_true.shape)==2:
            correct_confidence=np.sum(y_pred_clipped*y_true,axis=1)
        negative_log_likelihood=-np.log(correct_confidence)
        return negative_log_likelihood

class Activation_SoftMax_Loss_Catorgorical_Cross_Entropy():
    def __init__(self):
        self.activation=Activation_SoftMax()
        self.loss=Loss_CategoricalCross_Entropy()
    def forward(self,inputs,y_true):
        self.activation.forward(inputs)
        self.output=self.activation.output
        return self.loss.calculate(self.output,y_true)
    def backward(self,dvalues,y_true):
        samples=len(dvalues)
        if len(y_true.shape)==2:
            y_true=np.argmax(y_true,axis=1)
        self.dinputs=dvalues.copy()
        self.dinputs[range(samples),y_true] -=1
        self.dinputs=self.dinputs/samples
    
class Optimizer_GD:
    def __init__(self,learning_rate=0.01):
        self.learning_rate=learning_rate
    def update_params(self,layer):
        layer.weights += -self.learning_rate*layer.dweights
        layer.bias += -self.learning_rate*layer.dbias

In [ ]:
dense1=Layer_Dense(784,128)
activation1=Activation_RelU()
dense2=Layer_Dense(128,10)
loss_activation=Activation_SoftMax_Loss_Catorgorical_Cross_Entropy()
optimizer=Optimizer_GD()
for epoch in range(10001):
    dense1.forward(X)
    activation1.forward(dense1.output)
    dense2.forward(activation1.output)
    loss=loss_activation.forward(dense2.output,y)

    predictions=np.argmax(loss_activation.output,axis=1)
    accuracy=np.mean(predictions==y)
    if not epoch %100==0:
        print(f'epoch: {epoch}, ' +
              f'acc: {accuracy :.3f}, ' +
              f'loss: {loss:.3f}')
    loss_activation.backward(loss_activation.output,y)
    dense2.backward(loss_activation.dinputs)
    activation1.backward(dense2.dinputs)
    dense1.backward(activation1.dinputs)

    optimizer.update_params(dense1)
    optimizer.update_params(dense2)
    